In [1]:
import os
import pandas as pd

os.chdir(r'stanfordSentimentTreebank')

In [2]:
## Loading reviews
sents = pd.read_csv('datasetSentences.txt',sep = "\t")

alltog = '.'.join(sents.sentence.str.lower().to_list())
sent_chars = set(alltog)

In [3]:
## Loading Phrases
phrs = pd.read_csv('dictionary.txt', sep = "|", header = None, names= ['phrase', 'phrase id'] )
phrstog = '.'.join(phrs.phrase.str.lower().to_list())
phrs_chars = set(phrstog)

In [4]:
sent_chars - phrs_chars

{'¡', '¢', '£', '¦', '§', '¨', '©', '\xad', '¯', '±', '³', '´', '¶', '»', '¼'}

In [5]:
replacements = {        '-LRB-' : '(',
        '-RRB-' : ')',
        '\xa0' : ' ',
        '\xc2' : '',
        '\xc3\x83\xc2\xa0' : 'a',
        'à' : 'a',
        'Â' : '',
        'â' : 'a',
        'ã' : 'a',
        'Ã¡' : 'a',
        'Ã¢' : 'a',
        'Ã£' : 'a',
        'Ã¦' : 'ae',
        'Ã§' : 'c',
        'Ã¨' : 'e',
        'Ã©' : 'e',
        'Ã­' : 'i',
        'Ã¯' : 'i',
        'Ã±' : 'n',
        'Ã³' : 'o',
        'Ã´' : 'o',
        'Ã¶' : 'o',
        'Ã»' : 'u',
        'Ã¼' : 'u',
        'æ' : 'ae',
        'ç' : 'c',
        'è' : 'e',
        'é' : 'e',
        'í' : 'i',
        'ï' : 'i',
        'ñ' : 'n',
        'ó' : 'o',
        'ô' : 'o',
        'ö' : 'o',
        'û' : 'u',
        'ü' : 'u'}

In [6]:
for char_ in replacements.keys():
    sents.sentence = sents.sentence.str.replace(char_,replacements[char_])
    phrs.phrase = phrs.phrase.str.replace(char_,replacements[char_])
    
sents.sentence = sents.sentence.str.lower()
phrs.phrase = phrs.phrase.str.lower()

In [7]:
sents = pd.merge(sents, phrs, left_on = 'sentence', right_on = 'phrase', how = 'left')


In [8]:
## Target Values
sentiments = pd.read_csv('sentiment_labels.txt', sep = "|")
sentiments.head()
sents = pd.merge(sents, sentiments, how = 'left', left_on = 'phrase id', right_on = 'phrase ids')

## Train/Test Splits
splts = pd.read_csv('datasetSplit.txt')
sents = pd.merge(sents, splts, on = 'sentence_index', how = 'left')

In [9]:
sents = sents[['sentence','sentiment values', 'splitset_label']]

In [16]:
sents['label5'] = pd.cut(sents['sentiment values'], bins = 5)
sents['label25'] = pd.cut(sents['sentiment values'], bins = 25)

In [18]:
sents['label25'].nunique()

25